In [1]:
# 在 main.py 中测试
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())


__Python VERSION: 3.11.11 (main, Dec 11 2024, 16:28:39) [GCC 11.2.0]
__pyTorch VERSION: 2.6.0+cu124
__CUDA VERSION
__CUDNN VERSION: 90100
__Number CUDA Devices: 8
__Devices
index, name, driver_version, memory.total [MiB], memory.used [MiB], memory.free [MiB]
0, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 2501 MiB, 21752 MiB
1, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 4 MiB, 24249 MiB
2, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 4 MiB, 24249 MiB
3, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 4 MiB, 24249 MiB
4, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 4 MiB, 24249 MiB
5, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 4 MiB, 24249 MiB
6, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 4 MiB, 24249 MiB
7, NVIDIA GeForce RTX 3090, 550.142, 24576 MiB, 4 MiB, 24249 MiB
Active CUDA Device: GPU 0
Available devices  8
Current cuda device  0


In [2]:
import triton
import triton.runtime

print(triton.runtime.driver.active)

In [3]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))  # 显示第一张卡的型号

8
NVIDIA GeForce RTX 3090


In [4]:
import triton
import triton.language as tl
import torch

@triton.jit
def add_kernel(x_ptr, y_ptr, output_ptr, n, BLOCK_SIZE: tl.constexpr):
    pid = tl.program_id(axis=0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    mask = offsets < n
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)
    tl.store(output_ptr + offsets, x + y, mask=mask)

# 测试用例
n = 1024
x = torch.randn(n, device='cuda')
y = torch.randn(n, device='cuda')
output = torch.empty_like(x)

add_kernel[(n // 128,)](x, y, output, n, BLOCK_SIZE=128)
print("Triton kernel ran successfully. Output sum:", output.sum().item())

Triton kernel ran successfully. Output sum: 5.831356048583984
